# Notebook Imports

In [1]:
import pandas as pd
import numpy as np

In [43]:
TRAINING_DATA_FILE = 'SpamData/02_Training/train-data.txt'
TEST_DATA_FILE = 'SpamData/02_Training/test-data.txt'

TOKEN_SPAM_PROB_FILE = 'SpamData/03_Testing/prob-spam.txt'
TOKEN_HAM_PROB_FILE = 'SpamData/03_Testing/prob-nonspam.txt'
TOKEN_ALL_PROB_FILE = 'SpamData/03_Testing/prob-all-tokens.txt'

TEST_FEATURE_MATRIX = 'SpamData/03_Testing/test-features.txt'
TEST_TARGET_FILE = 'SpamData/03_Testing/test-target.txt'
VOCAB_SIZE = 2500

# Read and Load features from .txt files in to  numpy array

In [3]:
sparse_train_data = np.loadtxt(TRAINING_DATA_FILE,delimiter = ' ',dtype=int)
sparse_test_data = np.loadtxt(TEST_DATA_FILE,delimiter = ' ',dtype=int)

In [4]:
sparse_train_data[:5]

array([[ 0,  2,  1,  1],
       [ 0,  3,  1,  2],
       [ 0,  4,  1,  1],
       [ 0,  7,  1,  3],
       [ 0, 11,  1,  1]])

In [5]:
print('nr of rows in training file',sparse_train_data.shape[0])
print('nr of rows in testing file',sparse_test_data.shape[0])
print('nr of emails in training file',np.unique(sparse_train_data[:,0]).size)
print('nr of emails in testing file',np.unique(sparse_test_data[:,0]).size)

nr of rows in training file 258348
nr of rows in testing file 117777
nr of emails in training file 4013
nr of emails in testing file 1724


## How to create an empty Dataframe

In [6]:
column_names = ['DOC_ID'] + ['CATEGORY'] + list(range(0,VOCAB_SIZE))

In [7]:
len(column_names)

2502

In [8]:
index_names = np.unique(sparse_train_data[:,0])
index_names

array([   0,    1,    2, ..., 5791, 5794, 5795])

In [9]:
full_train_data = pd.DataFrame(index = index_names,columns = column_names)
full_train_data.fillna(value=0,inplace=True)

In [10]:
full_train_data.head()

,DOC_ID,CATEGORY,0,1,2,3,4,5,6,7,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Create a Full matrix from sparse matrix

In [11]:
def make_full_matrix(sparse_matrix,nr_words,doc_idx = 0,word_idx = 1, cat_idx = 2, freq_idx = 3):
    """
    Form a Full matrix from a sparse matrix. Return a pandas Dataframe.
    Keyword Arguements:
    sparse_matrix -- Numpy Array
    nr_words -- size of the vocabulary. Total number of tokens.
    doc_idx -- postion of the document id in the sparse matrix. Default: 1st column
    word_idx -- postion of the word id in the sparse matrix. Default: 2nd column
    cat_idx -- postion of the label (spam is 1,nonspam is 0). Default: 3rd column
    freq_idx -- postion of the occurance of word in the sparse matrix. Default: 4th column
    """
    
    column_names = ['DOC_ID'] + ['CATEGORY'] + list(range(0,VOCAB_SIZE))
    doc_id_names = np.unique(sparse_matrix[:,0])
    full_matrix = pd.DataFrame(index = doc_id_names,columns = column_names)
    full_matrix.fillna(value=0,inplace=True)
    
    for i in range(sparse_matrix.shape[0]):
        doc_nr = sparse_matrix[i][doc_idx]
        word_id = sparse_matrix[i][word_idx]
        label = sparse_matrix[i][cat_idx]
        occurance = sparse_matrix[i][freq_idx]
        
        full_matrix.at[doc_nr,'DOC_ID'] = doc_nr
        full_matrix.at[doc_nr,'CATEGORY'] = label
        full_matrix.at[doc_nr, word_id] = occurance
        
    full_matrix.set_index('DOC_ID',inplace=True)
    return full_matrix

In [12]:
%%time
full_train_data = make_full_matrix(sparse_train_data,VOCAB_SIZE)

Wall time: 20.6 s


In [13]:
full_train_data.shape

(4013, 2501)

In [14]:
full_train_data.head()

,CATEGORY,0,1,2,3,4,5,6,7,8,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,1,0,0,1,2,1,0,0,3,0,...,0,0,0,0,0,0,0,0,0,0
1,1,7,1,2,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,1,6,1,1,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,1,6,0,0,2,4,0,3,14,0,...,0,0,0,0,0,0,0,0,0,0
4,1,5,1,2,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


# Training the naive bayes model
## Calculating the probability of spam

In [15]:
full_train_data.CATEGORY.size

4013

In [16]:
full_train_data.CATEGORY.sum()

1248

In [17]:
prob_spam = full_train_data.CATEGORY.sum() / full_train_data.CATEGORY.size
print('the probability of spam is ',prob_spam)

the probability of spam is  0.310989284824321


## Total no of Words / Tokens

In [18]:
full_train_features = full_train_data.loc[:,full_train_data.columns != 'CATEGORY']
full_train_features.head()

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,0,0,1,2,1,0,0,3,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,1,2,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,1,1,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,6,0,0,2,4,0,3,14,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,1,2,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
email_lengths = full_train_features.sum(axis=1)
email_lengths.shape

(4013,)

In [20]:
email_lengths[:5]

DOC_ID
0     87
1     53
2     40
3    183
4     43
dtype: int64

In [21]:
total_wc = email_lengths.sum()
total_wc

429238

## Number of tokens in spam and ham emails

In [22]:
spam_lengths = email_lengths[full_train_data.CATEGORY == 1]
spam_lengths.shape

(1248,)

In [23]:
spam_wc = spam_lengths.sum()
spam_wc

176330

In [24]:
ham_lengths = email_lengths[full_train_data.CATEGORY == 0]
ham_lengths.shape

(2765,)

In [25]:
nonspam_wc = ham_lengths.sum()
nonspam_wc

252908

In [26]:
print('Avg no of words in spam emails {:.0f}'.format(spam_wc/spam_lengths.shape[0]))
print('Avg no of words in nonspam emails {:.0f}'.format(nonspam_wc/ham_lengths.shape[0]))

Avg no of words in spam emails 141
Avg no of words in nonspam emails 91


## Summing the tokens occuring in spam

In [27]:
full_train_features.shape

(4013, 2500)

In [28]:
train_spam_tokens = full_train_features.loc[full_train_data.CATEGORY==1]
train_spam_tokens.head()

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,0,0,1,2,1,0,0,3,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,1,2,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,1,1,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,6,0,0,2,4,0,3,14,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,1,2,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [29]:
train_spam_tokens.tail()

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
1885,1,0,0,2,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1887,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1889,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1890,2,0,0,0,1,0,0,2,0,0,...,0,0,0,0,0,0,0,0,0,0
1895,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [30]:
train_spam_tokens.shape

(1248, 2500)

In [31]:
summed_spam_tokens = train_spam_tokens.sum(axis=0) + 1
summed_spam_tokens.shape

(2500,)

In [32]:
summed_spam_tokens.tail()

2495    22
2496     2
2497     4
2498    26
2499     1
dtype: int64

## Summing the tokens occuring in ham

In [33]:
train_ham_tokens = full_train_features.loc[full_train_data.CATEGORY == 0]
summed_ham_tokens = train_ham_tokens.sum(axis=0) + 1
summed_ham_tokens.shape

(2500,)

In [34]:
summed_ham_tokens.tail()

2495     1
2496    23
2497    36
2498     3
2499    25
dtype: int64

In [35]:
train_ham_tokens[2499].sum() + 1

25

## P(Token | Spam) - Probability that a Token occurs given the Email is Spam

In [37]:
prob_tokens_spam = summed_spam_tokens / (spam_wc +VOCAB_SIZE)
prob_tokens_spam[:5]

0    0.012185
1    0.005228
2    0.006805
3    0.011307
4    0.006817
dtype: float64

In [38]:
prob_tokens_spam.sum()

1.0000000000000002

## P(Token | Ham) - Probability that a Token occurs given the Email is Spam


In [39]:
prob_tokens_nonspam = summed_ham_tokens / (nonspam_wc + VOCAB_SIZE)
prob_tokens_nonspam.sum()

1.0

# P(Token) - Probabilty that a token occurs

In [40]:
prob_tokens_all = full_train_features.sum(axis=0) / total_wc
prob_tokens_all.sum()

1.0

# Save the Trained Model

In [42]:
np.savetxt(TOKEN_SPAM_PROB_FILE,prob_tokens_spam)
np.savetxt(TOKEN_HAM_PROB_FILE,prob_tokens_nonspam)
np.savetxt(TOKEN_ALL_PROB_FILE,prob_tokens_all)

# Prepare Test data

In [44]:
sparse_test_data.shape

(117777, 4)

In [45]:
%%time
full_test_data = make_full_matrix(sparse_test_data,nr_words = VOCAB_SIZE)

Wall time: 8.85 s


In [46]:
X_test = full_test_data.loc[:,full_test_data.columns != 'CATEGORY']
y_test = full_test_data.CATEGORY

In [47]:
np.savetxt(TEST_TARGET_FILE,y_test)
np.savetxt(TEST_FEATURE_MATRIX,X_test)